# Importaciones

In [ ]:
%%capture
!%pip install mne
!%pip install --upgrade mne
!%pip install -U mne

In [ ]:
from zipfile import ZipFile
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import mne
import seaborn as sns
from mne import time_frequency
from sklearn.metrics import accuracy_score, recall_score
from tensorflow.keras.metrics import Recall
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Descomprimir, Leer, Convetir a MNE y Categorizar los datos

#unzip the files
data = ZipFile('EEGs_Guinea-Bissau.zip')
data2 = ZipFile('EEGs_Nigeria.zip')
data.extractall()
data2.extractall()

In [ ]:
meta_df=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_guineabissau.csv')
meta_df2 = pd.read_csv('https://zenodo.org/record/1252141/files/metadata_nigeria.csv')
meta_df.head()

,subject.id,Group,Eyes.condition,Remarks,recordedPeriod,startTime
0,1,Epilepsy,closed-3min-then-open-2min,by 45s reposition electrodes,301,27/5/2020 14:33
1,2,Control,open-3min-then-closed-2min,NaN,309,26/5/2020 22:44
2,3,Epilepsy,closed-3min-then-open-2min,NaN,309,27/5/2020 14:26
3,4,Epilepsy,closed-3min-then-open-2min,"Green lights not shown, but good EEG traces",299,27/5/2020 15:23
4,5,Control,closed-3min-then-open-2min,NaN,302,23/5/2020 19:09


In [ ]:
#now i need to seprate Epilepsy vs Control subjects
EP_sub=meta_df['subject.id'][meta_df['Group']=='Epilepsy']
CT_sub=meta_df['subject.id'][meta_df['Group']=='Control']

In [ ]:
#now i need to seprate Epilepsy vs Control subjects
EP_sub2=meta_df2['subject.id'][meta_df2['Group']=='Epilepsy']
CT_sub2=meta_df2['subject.id'][meta_df2['Group']=='Control']

In [ ]:
#read csv files
Epilepsy2=[pd.read_csv('EEGs_Nigeria/signal-{}.csv.gz'.format(i), compression='gzip') for i in  EP_sub2]
Control2=[pd.read_csv('EEGs_Nigeria/signal-{}.csv.gz'.format(i), compression='gzip') for i in  CT_sub2]

In [ ]:
#read csv files
Epilepsy=[pd.read_csv('EEGs_Guinea-Bissau/signal-{}.csv.gz'.format(i), compression='gzip') for i in  EP_sub]
Control=[pd.read_csv('EEGs_Guinea-Bissau/signal-{}.csv.gz'.format(i), compression='gzip') for i in  CT_sub]

In [ ]:
Epilepsy[0].head()

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,...,4,4,4,4,4,4,4,4,4,4
1,2,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,...,4,4,4,4,4,4,4,4,4,4
2,3,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,...,4,4,4,4,4,4,4,4,4,4
3,4,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,...,4,4,4,4,4,4,4,4,4,4
4,5,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,...,4,4,4,4,4,4,4,4,4,4


In [ ]:
#remove non eeg channels
Epilepsy=[i.iloc[:,1:15] for i in  Epilepsy]
Control=[i.iloc[:,1:15] for i in  Control]

In [ ]:
#remove non eeg channels
Epilepsy2=[i.iloc[:,1:15] for i in  Epilepsy]
Control2=[i.iloc[:,1:15] for i in  Control]

In [ ]:
Epilepsy[0].head()

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564


In [ ]:
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    data.filter(l_freq=0.1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    epochs=epochs.drop_bad()

    return epochs

In [ ]:
%%capture
#Convert each dataframe to mne object
Epilepsy=[convertDF2MNE(i) for i in  Epilepsy]
Control=[convertDF2MNE(i) for i in  Control]

#concatenate the epochs
Epilepsy_epochs=mne.concatenate_epochs(Epilepsy)
Control_epochs=mne.concatenate_epochs(Control)

In [ ]:
%%capture
#Convert each dataframe to mne object
Epilepsy2=[convertDF2MNE(i) for i in  Epilepsy2]
Control2=[convertDF2MNE(i) for i in  Control2]

#concatenate the epochs
Epilepsy_epochs2=mne.concatenate_epochs(Epilepsy2)
Control_epochs2=mne.concatenate_epochs(Control2)

In [ ]:
Epilepsy_group=np.concatenate([[i]*len(Epilepsy[i]) for i in range(len(Epilepsy))])#create a list of list where each sub list corresponds to subject_no
Control_group=np.concatenate([[i]*len(Control[i]) for i in range(len(Control))])#create a list of list where each sub list corresponds to subject_no

Epilepsy_label=np.concatenate([[0]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_label=np.concatenate([[1]*len(Control[i]) for i in range(len(Control))])

In [ ]:
Epilepsy_group2=np.concatenate([[i]*len(Epilepsy2[i]) for i in range(len(Epilepsy2))])#create a list of list where each sub list corresponds to subject_no
Control_group2=np.concatenate([[i]*len(Control2[i]) for i in range(len(Control2))])#create a list of list where each sub list corresponds to subject_no

Epilepsy_label2=np.concatenate([[0]*len(Epilepsy2[i]) for i in range(len(Epilepsy2))])
Control_label2=np.concatenate([[1]*len(Control2[i]) for i in range(len(Control2))])

In [ ]:
#combine data
data=mne.concatenate_epochs([Epilepsy_epochs,Control_epochs])
group=np.concatenate((Epilepsy_group,Control_group))
label=np.concatenate((Epilepsy_label,Control_label))
print(len(data),len(group),len(label))

Not setting metadata
7456 matching events found
No baseline correction applied
7456 7456 7456


In [ ]:
#combine data
data2=mne.concatenate_epochs([Epilepsy_epochs2,Control_epochs2])
group2=np.concatenate((Epilepsy_group2,Control_group2))
label2=np.concatenate((Epilepsy_label2,Control_label2))
print(len(data2),len(group2),len(label2))

Not setting metadata
7456 matching events found
No baseline correction applied
7456 7456 7456


# Extraer características de potencia del espectro de señales

In [ ]:
def eeg_power_band(epochs):
    # Define las bandas de frecuencia específicas
    FREQ_BANDS = {"delta": [0.5, 4],
                  "theta": [4, 8],
                  "alpha": [8, 16],
                  "beta": [16, 32],
                  "gamma": [32, 64]}

    # Compute la PSD usando el método de Welch
    psds, freqs = mne.time_frequency.psd_array_welch(epochs.get_data(copy=False), sfreq=epochs.info['sfreq'], fmin=0.5, fmax=45, n_fft=256, window='hamming')

    # Normaliza las PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    # Para cada banda de frecuencia, calcula la media de la PSD en esa banda
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band)

    return np.concatenate(X, axis=1)


In [ ]:
features=[]
for d in range(len(data)):#get features from each epoch and save in a list
    print(d)
    features.append(eeg_power_band(data[d]))

Se truncaron las últimas líneas 5000 del resultado de transmisión.
4956
Effective window size : 2.000 (s)
4957
Effective window size : 2.000 (s)
4958
Effective window size : 2.000 (s)
4959
Effective window size : 2.000 (s)
4960
Effective window size : 2.000 (s)
4961
Effective window size : 2.000 (s)
4962
Effective window size : 2.000 (s)
4963
Effective window size : 2.000 (s)
4964
Effective window size : 2.000 (s)
4965
Effective window size : 2.000 (s)
4966
Effective window size : 2.000 (s)
4967
Effective window size : 2.000 (s)
4968
Effective window size : 2.000 (s)
4969
Effective window size : 2.000 (s)
4970
Effective window size : 2.000 (s)
4971
Effective window size : 2.000 (s)
4972
Effective window size : 2.000 (s)
4973
Effective window size : 2.000 (s)
4974
Effective window size : 2.000 (s)
4975
Effective window size : 2.000 (s)
4976
Effective window size : 2.000 (s)
4977
Effective window size : 2.000 (s)
4978
Effective window size : 2.000 (s)
4979
Effective window size : 2.000 (

In [ ]:
features=np.concatenate(features)

In [ ]:
features2=[]
for d in range(len(data2)):#get features from each epoch and save in a list
    print(d)
    features2.append(eeg_power_band(data2[d]))

In [ ]:
features2=np.concatenate(features2)

# Ventanas Temporales

In [ ]:
def create_sliding_window(features, labels, window_size, step_size):
    segments = []
    window_labels = []

    # Asegurarse de que las etiquetas son de la misma longitud que las características
    if len(features) != len(labels):
        raise ValueError("La longitud de las características y las etiquetas debe coincidir")

    # Crear segmentos usando la ventana deslizante
    for start_pos in range(0, len(features) - window_size + 1, step_size):
        end_pos = start_pos + window_size
        segment = features[start_pos:end_pos]
        segments.append(segment)

        # Aquí suponemos que el label de cada ventana es el label del último punto de datos en la ventana
        window_labels.append(labels[end_pos - 1])

    return np.array(segments), np.array(window_labels)

In [ ]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
labels_encoded = to_categorical(label)

In [ ]:
scaler = StandardScaler()
features_scaled2 = scaler.fit_transform(features2)
labels_encoded2 = to_categorical(label2)

In [ ]:
window_size = 256  # 2 segundos con 128 Hz de frecuencia de muestreo
step_size = 128    # 1 segundo con 128 Hz de frecuencia de muestreo

# Aplicar ventana deslizante
X, y = create_sliding_window(features_scaled, labels_encoded, window_size, step_size)

In [ ]:

# Aplicar ventana deslizante
X2, y2 = create_sliding_window(features_scaled2, labels_encoded2, window_size, step_size)

# Experimentar con Nigeria de validación

In [ ]:
n_epochs = len(data2)
n_channels = data2.info['nchan']
n_time_points = data2.get_data(copy=False).shape[2]

# Calculate and display summary statistics for EEG data
eeg_data2 = data2.get_data(copy=False)  # Get the EEG data
mean_data2 = np.mean(eeg_data2, axis=(0, 2))  # Calculate the mean across epochs and time points
std_data2 = np.std(eeg_data2, axis=(0, 2))    # Calculate the standard deviation

In [ ]:
print("Number of Epochs:", n_epochs2)
print("Number of Channels:", n_channels2)
print("Number of Time Points:", n_time_points2)
print("Mean EEG Data:", mean_data2)
print("Standard Deviation of EEG Data:", std_data2)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=label2)
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.show()